In [1]:
import torch
from torch import nn
import torchvision.models as models
import torchvision
from tqdm.notebook import tqdm
import os
import pandas as pd
import numpy as np

In [2]:
dataset_path = "/content/drive/MyDrive/competitions/digit_recognizer/data" # @param {type:"string"}
batch_size = 1024
test_batch_size = 1
n_epoch = 256
#lr_init = 0.01

if not os.path.exists(dataset_path): # check dataset_path
  print('invalid dataset_path')


In [3]:
class datasets_from_csv():
  def __init__(self, dataset_file, idx=[0]): # read digit data from a csv file
    data = pd.read_csv(dataset_file)
    
    if 'label' in data: # if the dataset is train data
      self.dataset_type = 'train'
      self.inputs = torch.tensor(data.drop('label', axis=1).values).reshape([-1,1,28,28])/255
      self.target = torch.tensor(data['label'].values)
      if len(idx) > 1:
        self.inputs = self.inputs[idx[0], idx[1]]
        self.target = self.target[idx[0], idx[1]]
    else: # if the dataset is test data
      self.dataset_type = 'test'
      self.inputs = torch.tensor(data.values).reshape([-1,1,28,28])/255
      if len(idx) > 1:
        self.inputs = self.inputs[idx[0], idx[1]]

  def __getitem__(self, idx):
    if self.dataset_type == 'train':
      return (self.inputs[idx], self.target[idx])
    else:
      return self.inputs[idx]
  
  def __len__(self):
    return len(self.inputs)
    

def make_dataloaders(train_dataset, val_dataset, test_dataset):
  trainloader = torch.utils.data.DataLoader(train_dataset, pin_memory=True, batch_size=batch_size, shuffle=True, drop_last=False)
  valloader = torch.utils.data.DataLoader(train_dataset, pin_memory=True, batch_size=test_batch_size, shuffle=True, drop_last=False)
  testloader = torch.utils.data.DataLoader(test_dataset, pin_memory=True, batch_size=test_batch_size, shuffle=False, drop_last=False)

  return trainloader, valloader, testloader

In [4]:
class _resblock(nn.Module):
  def __init__(self, input_channel, output_channel, kernel_size=3, stride=1, padding=1):
    super(_resblock, self).__init__()
    self.conv1 = nn.Conv2d(input_channel, output_channel, kernel_size, 1, padding)
    self.bn1 = nn.BatchNorm2d(output_channel)
    self.relu = nn.ReLU()
    self.conv2 = nn.Conv2d(output_channel, output_channel, kernel_size, stride, padding)
    self.bn2 = nn.BatchNorm2d(output_channel)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)

    return x

class ResNet(nn.Module):
  def __init__(self, input_channel=1, output_channel=10):
    super(ResNet, self).__init__()
    self.input_channel = input_channel
    self.output_channel = output_channel

    self.resblock1 = _resblock(1,  32, 3, 1)
    self.resblock2 = _resblock(32, 32, 3, 1)
    self.resblock3 = _resblock(32, 64, 3, 2)
    self.resblock4 = _resblock(64, 64, 3, 1)
    self.resblock5 = _resblock(64, 128, 3, 2)
    self.resblock6 = _resblock(128, 128, 3, 1)

    self.conv1 = nn.Conv2d(32, 64, 1)
    self.conv2 = nn.Conv2d(64, 128, 1)

    self.fc = nn.Linear(128, 10)
    self.avg_pool = nn.AvgPool2d(2, 2)

    self.global_avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.relu = nn.ReLU()

  
  def forward(self, inputs):
    x = self.relu(self.resblock1(inputs)) + inputs
    x = self.relu(self.resblock2(x)) + x
    x = self.relu(self.resblock3(x)) + self.conv1(self.avg_pool(x))
    x = self.relu(self.resblock4(x)) + x
    x = self.relu(self.resblock5(x)) + self.conv2(self.avg_pool(x))
    x = self.relu(self.resblock6(x)) + x

    x = self.global_avgpool(x)
    x = self.fc(x.reshape(-1, 128))

    return x.reshape(-1, self.output_channel)


In [5]:
def train(model, train_loader, val_loader, n_epoch, lossfn=nn.CrossEntropyLoss()):

  optimizer = torch.optim.Adam(model.parameters())
  pbar_epoch = tqdm(range(n_epoch))

  for epoch in pbar_epoch:
    train_loss = []
    val_loss = []
    cnt = 0
    pbar_epoch.set_description(f'[Epoch : {epoch}/{n_epoch}')
    pbar_trainloader = tqdm(enumerate(train_loader), total=len(train_loader), leave=False, desc='train')
    pbar_valloader = tqdm(enumerate(train_loader), total=len(train_loader), leave=False, desc='validation')

    for i, data in pbar_trainloader:
      optimizer.zero_grad()
      inputs, target = data
      outputs = model(inputs) # estimation
      loss = lossfn(outputs, target) # calculate loss score
      train_loss.append(loss.detach()) # to keep loss score
      loss.backward()
      optimizer.step()
    
    with torch.no_grad():
      for i, data in pbar_trainloader:
        inputs, target = data
        outputs = model(inputs) # estimation
        loss = lossfn(outputs, target) # calculate loss score
        val_loss.append(loss.detach()) # to keep loss score
    

    avg_train_loss = np.average(np.array(train_loss))
    avg_val_loss = np.average(np.array(val_loss))
    pbar_epoch.set_postfixt(train_loss=avg_train_loss, val_loss=avg_val_loss)

In [6]:
train_data_file = os.path.join(dataset_path, 'train.csv')
test_data_file = os.path.join(dataset_path, 'test.csv')

train_dataset = datasets_from_csv(train_data_file, [0, 37800])
val_dataset = datasets_from_csv(train_data_file, [37800, -1])
test_dataset = datasets_from_csv(test_data_file)

train_loader, val_loader, test_loader = make_dataloaders(train_dataset,
                                                         val_dataset, 
                                                         test_dataset)

In [7]:
model = ResNet()

train(model, train_loader, val_loader, 4)

  0%|          | 0/4 [00:00<?, ?it/s]

train:   0%|          | 0/1 [00:00<?, ?it/s]

validation:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
def test(model, test_loader):
  output = []
  with torch.no_grad():
    for data in tqdm(test_loader):
      out = np.array(model(data).detach()).flatten()
      out = out.argmax()
      output.append(out)

  return np.array(output)

In [ ]:
submission = test(model, test_loader)

In [ ]:
submission_df = pd.DataFrame(np.array([np.arange(len(submission)), submission]).transpose(), columns=['ImageID', 'Label'])
submission_df.to_csv(os.path.join(dataset_path, 'submission.csv'))

In [24]:
train_dataset[:int(len(train_dataset)*0.9)][1].shape

torch.Size([37800])

In [26]:
for data in train_loader:
  print(data)
  break

RuntimeError: ignored

(tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]), tensor([1, 0, 1]))